# Alliance of Consumer Complaints Using NLP

In [1]:
# Import Libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
# To take bareframe from kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [3]:
# This package i donta have in my ssystem
# Install libraries if you want and restart the karnal
#pip install torchvision

In [4]:
# Libraries for Natural Language Processing
import nltk # Nural Language tool kit
from nltk.stem.porter import * # Stem
from torch.nn import * # Torch package
from torch.optim import * 
import numpy as np 
import pandas as pd
import torch,torchvision # Import tochvision
import random # Import random
from tqdm import * # TQDM
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [7]:
# Do Tocanisation
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abc\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [8]:
# Ex of Tokenisation
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [9]:
tokenize('$1000')

['$', '1000']

In [10]:
# Ex Seam the word
def stem(word):
    return stemmer.stem(word.lower())

In [11]:
stem('organic')

'organ'

In [12]:
# Bag the words
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words))
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [13]:
bag_of_words(['hi'],['hi','hey','hi'])

array([1., 0., 1.])

In [15]:
# Load the Data
data = pd.read_csv('complaints_processed.csv')[:5000]

In [ ]:
# After looking description i conclude that we dont need to work on the reli on null values

In [16]:
# Drop the null values
data.dropna(inplace=True)

In [17]:
# Sampling of the datapoints
data = data.sample(frac=1)

In [18]:
data.drop(data.index[data['narrative'] == 'name'],inplace=True)

In [19]:
# We have two featurs in the data
# Indepandant veriable
X = data['narrative']

In [20]:
# Dependant Veriable
y = data['product']

In [21]:
# Packaging of data
# Initilizting the list
words = [] # For words
data = [] # For data
labels = {} # For labels-- Featurs
idx = 0 # Set index as 0

In [22]:
# Convert some of the major datapoints as numirical assignment
for X_batch,y_batch in zip(X,y):
    if y_batch not in list(labels.keys()):
        idx += 1
        labels[y_batch] = idx

In [23]:
# Print the labels
labels

{'retail_banking': 1,
 'debt_collection': 2,
 'credit_card': 3,
 'credit_reporting': 4,
 'mortgages_and_loans': 5}

In [24]:
# seam all the datapoinbts to make it interpritable 
# Simply process the words to make there bagages
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    words.extend(new_X)
    data.append([X_batch,np.eye(labels[y_batch] + 1,len(labels))[labels[y_batch]]])    

4997it [00:40, 122.52it/s]


In [25]:
# save itt in assign list
words = sorted(set(words))

In [26]:
# Suffle the datapoints
np.random.shuffle(data)

In [27]:
# Assign the X, Y to save those sorted values that we did before
X = []
y = []

In [28]:
# Now niniaturise the datapoints
# simply clearify the sentence
for sentence,tag in tqdm(data):
    X.append(bag_of_words(sentence,words))
    y.append(tag)

100%|██████████████████████████████████████████████████████████████████████████████| 4997/4997 [02:44<00:00, 30.40it/s]


In [29]:
# Import the model
from sklearn.model_selection import *

In [30]:
# Split the datapoints
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)

In [31]:
# Assign the divice type
device = 'cpu'

In [32]:
# assign those numirical datapoints to device
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [33]:
# Model evalution
def get_loss(model,X,y,criterion): # For loss
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

def get_accuracy(model,X,y): # For Accurecy
    preds = model(X)
    correct = 0
    total = 0
    for pred,y_batch in zip(preds,y):
        pred = int(torch.argmax(pred))
        y_batch = int(torch.argmax(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [34]:
# Initilied the model
class Model(Module):
    def __init__(self):
        super().__init__()
        self.activation = ReLU()
        self.iters = 10
        self.linear1 = Linear(len(words),512)
        self.linear2 = Linear(512,512)
        self.bn = BatchNorm1d(512)
        self.output = Linear(512,len(labels))
    
    def forward(self,X):
        preds = self.linear1(X)
        for _ in range(self.iters):
            preds = self.activation(self.bn(self.linear2(preds)))
        preds = self.output(preds)
        return preds

In [35]:
# Fit the model
model = Model().to(device)

In [36]:
# Set the Evalution
criterion = MSELoss()

In [37]:
# Optimized the model
optimizer = Adam(model.parameters(),lr=0.001)

In [38]:
# Set tyhe parameters
batch_size = 32
epochs = 100

In [40]:
#pip install wandb

  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21500 sha256=53adcf1123fcdc06b8c4b8d3231a58e5d32d3fe677d6c6f0c270680f1f66b85c
  Stored in directory: c:\users\abc\appdata\local\pip\cache\wheels\54\aa\01\724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Successfully built promise
Note: you may need to restart the kernel to use updated packages.


In [41]:
# Setup the db
# Give the project name
import wandb
PROJECT_NAME = 'Alliance-Consumer-Complaints-NLP'

In [43]:
# Deploy the model
torch.cuda.empty_cache()
for _ in tqdm(range(epochs)):
    torch.cuda.empty_cache()
    for i in range(0,len(X_train),batch_size):
        torch.cuda.empty_cache()
        X_batch = X_train[i:i+batch_size].to(device).float()
        y_batch = y_train[i:i+batch_size].to(device).float()
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
    torch.cuda.empty_cache()
torch.cuda.empty_cache()

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [27:48<00:00, 16.68s/it]


In [44]:
# Retrive all metrixes
get_loss(model,X_train,y_train,criterion),get_loss(model,X_test,y_test,criterion),get_accuracy(model,X_train,y_train),get_accuracy(model,X_test,y_test)

(0.0033273680601269007,
 0.05226670950651169,
 91.10000000000001,
 79.10000000000001)

In [45]:
# Save all implecations --1
torch.save(words,'words.pt')
torch.save(words,'words.pth')
torch.save(data,'data.pt')
torch.save(data,'data.pth')
torch.save(labels,'labels.pt')
torch.save(labels,'labels.pth')

In [46]:
# save all implications -- 2
torch.save(X_train,'X_train.pt')
torch.save(X_test,'X_test.pth')
torch.save(y_train,'y_train.pt')
torch.save(y_test,'y_test.pth')
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(model.state_dict(),'model-sd.pt')
torch.save(model.state_dict(),'model-sd.pth')
torch.save(X,'X.pt')
torch.save(X,'X.pth')
torch.save(y,'y.pt')
torch.save(y,'y.pth')

In [ ]:
# Learning never ends